In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# data overview
sample_submission = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv")


In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv("/kaggle/working/submission.csv",index=False)

# Second submission

## Libraries

In [ ]:
import zipfile
import pandas as pd
#import transformers

In [ ]:
def read_in_zip(name):
    with zipfile.ZipFile("data/commonlit-evaluate-student-summaries.zip") as z:
        with z.open(name) as f:
            df = pd.read_csv(f)
    return df

In [ ]:

alpaca_model = transformers.AutoModelForCausalLM.from_pretrained("/kaggle/input/alpaca/pytorch/7b-wdiff/1")
alpaca_tokenizer = transformers.AutoTokenizer.from_pretrained("/kaggle/input/alpaca/pytorch/7b-wdiff/1")

In [ ]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
from langchain.llms import HuggingFaceHub

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
question = summaries_train["text"][3]

template = """Student Summary: {question}

What is the quality in the last student summary? Answer <High>, <Medium> or <Low>"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
repo_id = "google/flan-t5-base"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

In [ ]:
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

print(llm_chain.run(question))

In [ ]:
name = "summaries_test.csv"
summaries_test = read_in_zip(name)

In [ ]:
summaries_test.shape

In [ ]:
summaries_test

In [ ]:
name = "summaries_train.csv"
summaries_train = read_in_zip(name)

In [ ]:
summaries_train.shape

In [ ]:
summaries_train.head()

In [ ]:
summaries_train["text"][0]

In [ ]:


from langchain.output_parsers import StructuredOutputParser, ResponseSchema
#from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
#from langchain.llms import OpenAI



In [ ]:
#llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5}
)

In [ ]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="student_resume", description="This the student resume provided by the user"),
    ResponseSchema(name="quality", description="This is your response, the resume quality")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:


# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (format_instructions)



In [ ]:
template = """
You will be given a student summary from a user.
Answer the student resume you were provided.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input=summaries_train["text"][0])

print(promptValue)

In [ ]:
llm_output = llm(promptValue)
llm_output

In [ ]:


output_parser.parse(llm_output)

